In [293]:
points = [
    [29.961542, 76.823127],
    [32.961542, 76.823127]
]
circle = {
        "center": [29.961542, 76.823127],
        "radius": 120,
        "strip_val": [0.5, 0.17, 0.06],
        "color": 'crimson',
        "trust_val": 69
}
radius = 100

In [294]:
import numpy as np
from numpy import arctan2,random,sin,cos,degrees
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils import *
from temp import *
import folium


In [232]:
a = {'lat': points[0][0], 'lon': points[0][1]}
b = {'lat': points[0][1], 'lon': points[1][1]}
dL = b['lon']-a['lon']
X = cos(b['lat'])* sin(dL)
Y = cos(a['lat'])*sin(b['lat']) - sin(a['lat'])*cos(b['lat'])* cos(dL)
bearing = arctan2(X,Y)

In [233]:
degrees(bearing), ((degrees(bearing)+360) % 360)

(-180.0, 180.0)

In [363]:
points = []
x, y = circle["center"]
x, y = [29.961264, 76.826217]
points.extend(PointsInCircum(x, y, circle['radius']*0.002, n=50))

In [364]:
circle['radius']*0.001

0.12

In [519]:
# xx, yy = zip(*points)
# temp = points + rectangle
points.extend(rectangle)
xx, yy = zip(*points)
min_x = min(xx)
min_y = min(yy)
max_x = max(xx)
max_y = max(yy)
bbox = [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]
bbox += [bbox[0]]

# lower_left = bbox[0]
# upper_right = bbox[2]
lower_left, upper_right = rectangle[0], rectangle[2]
grid = get_geojson_grid(upper_right, lower_left, n=11)

In [520]:
m = folium.Map(location=points[0], zoom_start=3, tiles="CartoDB dark_matter")
points = []
x, y = circle["center"]
x, y = [29.961264, 76.826217]
points.extend(PointsInCircum(x, y, circle['radius']*0.0015, n=100))
folium.PolyLine(points).add_to(m)

rectangle = [
    displace(x,y, 90, radius*1.25),
    displace(x1,y1, 90, radius),
    displace(x1,y1, -90, radius),
    displace(x,y, -90, radius*1.25)
]

rectangle += [rectangle[0]]
folium.PolyLine(rectangle).add_to(m)

In [521]:
trust = 59
mid = int(len(grid)**(1/2))//2
strips = [0.52, 0.223, 0.17][::-1]
total = 0
for i, geo_json in enumerate(grid):
#     color = plt.cm.Greens(i / len(grid))
#     strip_id = 1 - abs((mid-i//10)
    strip_id = mid-abs(mid-i//11)
    geo_json["prob"] = strips[strip_id//2] * trust
    total += geo_json["prob"]

In [522]:
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(geo_json["prob"]/100)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
#     popup = folium.Popup("example popup {}".format(abs(i-15)))
    popup = folium.Popup(f"row-{mid - i//11}-{geo_json['prob']}")
    gj.add_child(popup)

    m.add_child(gj)

In [523]:
m

In [ ]:
m = folium.Map(location=points[0], zoom_start=3, tiles="CartoDB dark_matter")


In [524]:
points = []
x, y = circle["center"]
x, y = [29.961264, 76.826217]
points.extend(PointsInCircum(x, y, circle['radius']*0.0015, n=100))

rectangle = [
    displace(x,y, 90, radius*1.25),
    displace(x1,y1, 90, radius),
    displace(x1,y1, -90, radius),
    displace(x,y, -90, radius*1.25)
]

rectangle += [rectangle[0]]
points.extend(rectangle)
folium.PolyLine(points).add_to(m)
folium.PolyLine(rectangle).add_to(m)

In [525]:
xx, yy = zip(*points)
min_x = min(xx)
min_y = min(yy)
max_x = max(xx)
max_y = max(yy)
bbox = [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]
bbox += [bbox[0]]

lower_left = bbox[0]
upper_right = bbox[2]
# lower_left, upper_right = rectangle[0], rectangle[2]
grid = get_geojson_grid(upper_right, lower_left, n=11)

In [526]:
for i, geo_json in enumerate(grid):
#     color = plt.cm.Reds(geo_json["prob"]/100)
    color = plt.cm.Reds(i/100)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("example popup {}".format(abs(i-15)))
#     popup = folium.Popup(f"row-{mid - i//11}-{geo_json['prob']}")
    gj.add_child(popup)

    m.add_child(gj)

In [528]:
m

In [577]:
m = folium.Map(location=points[0], zoom_start=3, tiles="CartoDB dark_matter")

In [581]:
trust = 59
mid = int(len(grid)**(1/2))//2
strips = [0.52, 0.223, 0.17][::-1]
total = 0
lat_steps = np.linspace(min_x, max_x, 4)
lat_steps = np.linspace(min_y, max_y, 4)

for _grid in grid:
    _points = _grid["features"][0]["geometry"]["coordinates"][0][:-1]
    xx, yy = zip(*_points)
    centroid = (sum(xx) / len(_points), sum(yy) / len(_points))
    lat, lon = centroid
    strip_id = check_range_rectangle(lat, lat_steps)

#     strip_id = mid-abs(mid-i//11)
    _grid["prob"] = strips[strip_id//2] * trust

In [584]:
total = sum(list(set(_grid["prob"] for _grid in grid)))

In [585]:
total

23.187

In [586]:
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(geo_json["prob"]/total)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("example popup {}".format(abs(i-15)))
#     popup = folium.Popup(f"row-{mid - i//11}-{geo_json['prob']}")
    gj.add_child(popup)

    m.add_child(gj)

In [587]:
m

In [568]:
def check_range_rectangle(lat, lat_steps):
    """
    Check in which range of the radius does the distance lie in
    """
#     print(lat, lat_steps)
    for i in range(len(lat_steps)-1):
        renge = (lat_steps[i], lat_steps[i+1])
        if renge[0] <= lat and lat <= renge[1]:
            return i
    return None